# 1 Data structures

## 1.a Matrix

A common approach is to represent vertices with 0s with implicit edges (left, right, down, up). 1s mean "no edge".

*Example:*

In [77]:
graph=[
    [0,1,1,0],
    [0,0,1,0],
    [1,1,0,0],
    [0,0,0,0],
]

## 1.b Adjacency matrix

In this case; M[r=i][c=j] means an edge exists from vertice *i* to vertice *j*. This structure is not that common in CS because it has a O(V<sup>2</sup>) space complexity, even though the graph has only *V* vertices.

## 1.c Adjacency list

In [78]:
class Vertice:
    def __init__(self, val):
        self.val = val
        self.neighbors = []

# 2 Matrix DFS

Question: count unique paths from top left to bottom right. Only 0s are allowed, no more than 1 visit per node for a given path.

## Architecture:

```
dfs(grid: list[list[int]], row: int, col: int, visited: set[tuple[int, int]]):

    # Base case: invalid
    if <OOB> or <Blocked> or <Visited>:
        return 0
    # Base case: invalid
    if <Reached>:
        return 1

    # Recursive call
    visit.add(row, col)
    count = 0
    count += dfs(<above|left|right|below>)
    visit.remove(row, col)
    
    return count
```

## Complexity

### Time complexity

Intuition: a path can "cover" the entire matrix, adding to `n*m` nodes. At each step of the paths, a recursive call is made to up to 4 nodes. Hence the worst-case time complexity is $O(4^{n.m})$.

### Space complexity

A given path is composed of up to `n*m` nodes, hence the size of the recursive call stack can be $O(n.m)$ in the worst case.

In [79]:
# DFS: implementation

def dfs(grid: list[list[int]], row: int = 0, col: int = 0, visited: set[tuple[int, int]] = set()):
    R, C = len(grid), len(grid[0])
    # OOB or blocked or visited
    base_case_invalid = min(row, col) < 0\
        or row >= R or col >= C\
        or grid[row][col] == 1\
        or (row, col) in visited

    # Reached target
    base_case_valid = ((row, col) == (R - 1, C - 1))

    if base_case_invalid:
        return 0

    if base_case_valid:
        return 1
    
    # Recursive calls
    visited.add((row, col))
    
    count = 0
    count += dfs(grid, row - 1, col, visited)
    count += dfs(grid, row + 1, col, visited)
    count += dfs(grid, row, col - 1, visited)
    count += dfs(grid, row, col + 1, visited)

    visited.remove((row, col))

    return count

from numpy import array
m = array([[0]*4 for _ in range(4)])
m[1,0] = m[1, 1] = m[3, 1] = m[2, 3] = 1
m

array([[0, 0, 0, 0],
       [1, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0]])

In [80]:
dfs(m)

2

# 3 Matrix BFS

Question: Length of shortest path from top left to bottom right.

## Architecture:

```
bfs():
    queue, visit = deque[(0, 0)], set[(0, 0)]
    length = 1

    while queue:
        length += 1
        for _ in [1..len(queue)]:

            row, col = queue.popleft()

            if <Reached>:
                return length

            for neighbor in neighbors(row, col):
                if <OOB> or <Blocked> or <Visited>:
                    continue
                queue.append(neighbor)
                visit.add(neighbor)         
```

## Complexity

### Time complexity

Worst-case scenario, all the nodes are visited exactly once (thanks to the maintained `set`), so the time complexity is $O(n.m)$

### Space complexity

All the created objects are $O(n.m)$.


In [81]:
# BFS: implementation

from collections import deque

def bfs(grid: list[list[int]]) -> int:
    queue = deque()
    visited = set()
    R, C = len(grid), len(grid[0])

    visited.add((0, 0))
    queue.append((0, 0))
    length = -1
    while queue:
        length += 1
        for _ in range(len(queue)):
            row, col = queue.popleft()
            if (row, col) == (R - 1, C - 1):
                return length
            
            deltas = [(1, 0), (-1, 0), (0, 1), (0, -1)]
            for dr, dc in deltas:
                neighbor = (row + dr, col + dc)

                invalid_neighbor = min(*neighbor) < 0 or neighbor[0] >= R or neighbor[1] >= C \
                    or grid[neighbor] == 1 \
                    or neighbor in visited
                if invalid_neighbor:
                    continue
                
                visited.add(neighbor) # Important to do ASAP: e.g. (1, 1) is a neighbor of both (0, 1) and (1, 0), which are neighbors of (0, 0)
                queue.append(neighbor)

In [82]:
bfs(m)

6

# 4 Adjacency lists

## 4.a Graph nodes

In [83]:
class GraphNode:
    def __init__(self, val, neighbors: list = []):
        self.val = val
        self.neighbors = neighbors

## 4.b Hash maps

In [84]:
# Example
adj = {"A": ["B", "C"], "B": ["A"]}

# Build an adjacency list from a list of directed edges
edges = [["A", "B"], ["B", "C"], ["B", "E"], ["C", "E"], ["E", "D"]]

# defaultdict is not that useful here (save 1 line)
# because a vertice that is only an arrival (e.g. "C") does not get added as key
adj = {}
for src, dst in edges:
    if src not in adj:
        adj[src] = []
    if dst not in adj:
        adj[dst] = []
    adj[src].append(dst)

adj

{'A': ['B'], 'B': ['C', 'E'], 'C': ['E'], 'E': ['D'], 'D': []}

## 4.c Traversal algorithms

In [85]:
# DFS

def dfs(adj: dict[str: list[str]], src: str, dst: str, visited: set[str] = set()) -> int:

    if src == dst:
        return 1
    
    if src in visited:
        return 0
    
    count = 0
    visited.add(src)
    for neighbor in adj[src]:
        count += dfs(adj, neighbor, dst, visited)
    visited.remove(src)
    return count

In [86]:
dfs(adj, "A", "E")

2

In [87]:
# BFS
from collections import deque
def bfs(adj: dict[str: list[str]], src: str, dst: str) -> int:
    queue = deque()
    visited = set()

    queue.append(src)
    visited.add(src)
    length = -1

    while queue:
        length += 1
        for _ in range(len(queue)):
            current = queue.popleft()
            if current == dst:
                return length
            for neighbor in adj[current]:
                if neighbor not in visited:
                    queue.append(neighbor)
                    visited.add(neighbor)
    return -1


In [88]:
bfs(adj, "A", "E")

2

# 5 Shortest path algorithms

## 5.a Dijkstra

Implementation: heapq does not support updating priority queue keys, so instead the same node can be pushed several times, with different keys. It is only marked as visited when it is popped from the queue for the first time (and in such a case, the key is guaranteed to be minimal).

In [89]:
import heapq

def dijkstra_with_set(n, edges: list[tuple[str, str, int]], src: str) -> dict[str: int]:
  # Convert edges to adjacency list
  adj_list = {v: [] for v in range(n)}
  for v, neighbor, distance in edges:
     adj_list[v].append((neighbor, distance))
  # Dijkstra
  visited, shortest = set(), {v: float("inf") for v in adj_list}
  heap = [(0, src)]
  while heap:
      src_to_v, v = heapq.heappop(heap)
      if v in visited:
          continue
      visited.add(v)
      shortest[v] = src_to_v
      for neighbor, v_to_neighbor in adj_list[v]:
          # if neighbor not in visited: # Optional, this can just reduce the size of the heap 
          heapq.heappush(heap, (src_to_v + v_to_neighbor, neighbor))
  return shortest

print(dijkstra_with_set(n=5, edges=[[0,1,10],[0,2,3],[1,3,2],[2,1,4],[2,3,8],[2,4,2],[3,4,5]], src=0))

{0: 0, 1: 7, 2: 3, 3: 9, 4: 5}


In [90]:
# Using the dictionary `shortest` as the set of visited nodes:

def dijkstra(n, edges: list[tuple[str, str, int]], src: str) -> dict[str: int]:
  # Convert edges to adjacency list
  adj_list = {v: [] for v in range(n)}
  for v, neighbor, distance in edges:
      adj_list[v].append((neighbor, distance))
  # Dijkstra
  shortest = {}
  heap = [(0, src)]
  while heap:
      src_to_v, v = heapq.heappop(heap)
      if v in shortest:
          continue
      shortest[v] = src_to_v
      for neighbor, v_to_neighbor in adj_list[v]:
          heapq.heappush(heap, (src_to_v + v_to_neighbor, neighbor))
  return shortest

print(dijkstra(n=5, edges=[[0,1,10],[0,2,3],[1,3,2],[2,1,4],[2,3,8],[2,4,2],[3,4,5]], src=0))

{0: 0, 2: 3, 4: 5, 1: 7, 3: 9}


## 5.b A*

# 6 Topological Sort

For Directed Acyclic Graphs:
* Directed: who comes first in undirected edge (u, v)?
* Acyclical: same problem, ordering nodes in a cycle is ambiguous

In [91]:
# Input graph
edges = [
    ("A", "B"),
    ("A", "C"),
    ("B", "D"),
    ("C", "E"),
    ("D", "F"),
    ("E", "F"),
    ("G", "H")
]

# Adjacency list
graph = {}
for src, dst in edges:
    if src not in graph:
        graph[src] = []
    if dst not in graph:
        graph[dst] = []
    graph[src].append(dst)

# Connected graph, knowing start node
def top_sort_with_start(graph: dict[str: list[str]], start_node: str) -> list[str]:
    res = []
    visited = set()

    def dfs_postorder(v: str) -> None:
        if v in visited:
            return
        visited.add(v)
        for nei in graph[v]:
            dfs_postorder(nei)
        res.append(v)
    
    dfs_postorder(start_node)
    res.reverse()
    return res

print(top_sort_with_start(graph=graph, start_node="A"))

# General case
def top_sort_no_cycle(graph: dict[str: list[str]]) -> list[str]:

    def dfs_postorder(v: str) -> None:
        if v in visited:
            return
        visited.add(v)
        for nei in graph[v]:
            dfs_postorder(nei)
        res.append(v)

    res, visited = [], set()
    for v in graph:
        dfs_postorder(v)
    res.reverse()
    return res

print(top_sort_no_cycle(graph))

# Topological sort with cycle detection

class CycleDetectedError(Exception):
    pass

def top_sort(graph: dict[str: list[str]]) -> list[str]:
    def dfs(v:str, path: set[str]) -> None:
        if v in path:
            raise CycleDetectedError("Could not run topological sort: cycle detected in input graph")
        if v in visited:
            return
        path.add(v)
        visited.add(v)
        for nei in graph[v]:
            dfs(nei, path)
        path.remove(v)
        topsort.append(v)
        return

    visited = set()
    topsort = []
    for v in graph:
        path = set()
        dfs(v, path)
    topsort.reverse()
    return topsort

print(top_sort(graph))

['A', 'C', 'E', 'B', 'D', 'F']
['G', 'H', 'A', 'C', 'E', 'B', 'D', 'F']
['G', 'H', 'A', 'C', 'E', 'B', 'D', 'F']


# 7 Cycle detection - Directed graphs

*Algrithms tested on leetcode's Course Schedule problem*

With directed graphs, the nodes must be marked two distinct ways, e.g. 'explored' or 'being_explored', as opposed to the usual `visited` set. This is necessary to avoid wrongly identifying a cycle in such a configuration: `{A: [B, C], B: [D], C: [D], D: []}`

## 7.a Using Topological Sort

See previous section

## 7.a Using standard DFS

In [92]:
# Check `being_explored` before `explored`!!!
# A node can be explored and not being_explored, but not being_explored and not explored!
# To avoid this mistake for sure the current vertex can be marked as explored only at the end of the procedure

def has_cycle_set(graph: dict[str: list[str]]) -> bool:
    
    def dfs(v: str) -> bool:
        if v in being_explored:
            return True
        if v in explored:
            return False
        explored.add(v)
        being_explored.add(v)
        for nei in graph[v]:
            if dfs(nei):
                return True
        being_explored.remove(v)
        return False

    for v in graph:
        explored, being_explored = set(), set()
        if dfs(v):
            return True
    return False

# Feels a bit less natural but some recursive calls are avoided
def has_cycle_set2(graph: dict[str: list[str]]) -> bool:
    
    def dfs(v: str) -> bool:
        explored.add(v)
        being_explored.add(v)
        back_edge = False
        for nei in graph[v]:
            if nei in being_explored:
                return True
            if nei in explored:
                continue
            back_edge = back_edge or dfs(nei)
        being_explored.remove(v)
        return back_edge

    for v in graph:
        explored, being_explored = set(), set()
        if dfs(v):
            return True
    return False

In [93]:
from enum import Enum

class Status(Enum):
    NOT_EXPLORED, EXPLORED, BEING_EXPLORED = range(1, 4)

def has_cycle_map(raph: dict[str: list[str]]) -> bool:

    def dfs(v: str) -> bool:
        status[v] = Status.BEING_EXPLORED
        res = False
        for neighbor in adj[v]:
            if status[neighbor] == Status.EXPLORED:
                continue
            if status[neighbor] == Status.BEING_EXPLORED:
                return True
            res = res or dfs(neighbor, status)
        status[v] = Status.EXPLORED
        return res

    for v in adj:
        status = {node: Status.NOT_EXPLORED for node in adj}
        if dfs(v, status):
            return True
    return False

# 8 Cycle detection - Undirected graphs

If the graph is connected, just count the number of edges E. If $E \geq V$, then there must be a cycle.  

Otherwise, the DFS method is simpler than for directed graphs because any encounter with a visited node means a cycle, except if the already visited node is the parent of the current node.

In [94]:
# Adjacency list
def build_graph(edges: list[tuple[str, str]]) -> dict[str: list[str]]:
    graph = {}
    for src, dst in edges:
        if src not in graph:
            graph[src] = []
        if dst not in graph:
            graph[dst] = []
        graph[src].append(dst)
        graph[dst].append(src)
    return graph
    
def has_cycle_undirected(graph: dict[str: list[str]]) -> bool:
    def dfs(v: str, p:str):
        visited.add(v)
        for nei in graph[v]:
            if nei == p:
                continue
            if nei in visited or dfs(nei, v):
                return True
        return False

    visited =set()
    for v in graph:
        if v in visited:
            continue
        if dfs(v, None):
            return True
    return False

acyclic = [
    ("A", "B"),
    ("A", "C"),
    ("C", "D"),
    ("D", "E"),
    ("D", "G"),
    ("E", "F")
]

cyclic_connected = [
    ("A", "B"),
    ("A", "C"),
    ("C", "D"),
    ("D", "E"),
    ("D", "F"),
    ("D", "G"),
    ("E", "F"),
    ("X", "Y"),
    ("Y", "Z")
]

cyclic_unconnected = [
    ("A", "B"),
    ("A", "C"),
    ("C", "D"),
    ("D", "E"),
    ("D", "F"),
    ("D", "G"),
    ("E", "F"),
    ("X", "Y"),
    ("Y", "Z")
]
graphs = [build_graph(edges) for edges in (acyclic, cyclic_connected, cyclic_unconnected)]
print([has_cycle_undirected(graph) for graph in graphs])

[False, True, True]


# 9 Union-Find a.k.a Disjoint-Set

Use-cases:
* Count connected components
* Detect cycles

*Both implementations tested using Leetcode's Redundant Connection*

## 9.a Core ideas

Input:
* ⚠️ Number of vertices: must be known upfront
* List of edges

Vanilla algorithm:
* Initially, the data structures is composed of all the nodes with no edges, except self-references: each node is its own parent
* For each edge, get the root parent ("representative") of each node, characterized by identity node = parent(node)
    * If the roots are the same, then the input graph contains a cycle
    * Else, connect the **roots** (the roots! Not the parents!)

In [1]:
from __future__ import annotations

class UnionFindNaive:
    def __init__(self: UnionFindNaive, n_vertices: int) -> None:
        self.parent = {vertex: vertex for vertex in range(1, n_vertices + 1)}

    def find(self: UnionFindNaive, v: int) -> int:
        while v != self.parent[v]:
            v = self.parent[v]
        return v
    
    def union(self: UnionFindNaive, edge: tuple[int, int]) -> bool:
        v1, v2 = edge
        root1, root2 = self.find(v1), self.find(v2)
        if root1 == root2:
            return False
        self.parent[root2] = root1
        return True

In [2]:
uf = UnionFindNaive(5)
uf.union((1, 4))
print(uf.parent)
uf.union((3, 4))
print(uf.parent)
uf.union((1, 3))

{1: 1, 2: 2, 3: 3, 4: 1, 5: 5}
{1: 3, 2: 2, 3: 3, 4: 1, 5: 5}


False

In [3]:
uf.parent

{1: 3, 2: 2, 3: 3, 4: 1, 5: 5}

## 9.b Heuristics that make Union-Find efficient

Quoting from [*Wikipedia*](https://en.wikipedia.org/wiki/Disjoint-set_data_structure)

### i) Union by size / Union by rank (=height)

> The choice of which node becomes the parent has consequences for the complexity of future operations on the tree. If it is done carelessly, trees can become excessively tall. For example, suppose that Union always made the tree containing x a subtree of the tree containing y. Begin with a forest that has just been initialized with elements 1, 2, 3, …, n and execute Union(1, 2), Union(2, 3), ..., Union(n - 1, n). The resulting forest contains a single tree whose root is n, and the path from 1 to n passes through every node in the tree. For this forest, the time to run Find(1) is O(n).

Hence, before merging two trees, the size (number of descendants including self) or the rank (upper bound of height) of both roots are compared. The root with the largest value becomes the parent of the other root.

*Note 1*: The *rank* of vertex V corresponds roughly to the *height* of the tree rooted at V. It is actually an upper bound of the rank: initially, the rank is equal to the height, but then applying a `.find()` operations may reduce the height of a given tree. The union by rank method avoids the overhead of maintaining heights for each `.find()` operation.

*Note 2*: only the size/ranks of the root is maintained through a `.union()` operation because

> It is clear from the above implementations that the size and rank of a node do not matter unless a node is the root of a tree. Once a node becomes a child, its size and rank are never accessed again.

### ii) Path compression

> Performing a Find operation presents an important opportunity for improving the forest. The time in a Find operation is spent chasing parent pointers, so a flatter tree leads to faster Find operations. When a Find is executed, there is no faster way to reach the root than by following each parent pointer in succession. However, the parent pointers visited during this search can be updated to point closer to the root. Because every element visited on the way to a root is part of the same set, this does not change the sets stored in the forest. But it makes future Find operations faster, not only for the nodes between the query node and the root, but also for their descendants. This updating is an important part of the disjoint-set forest's amortized performance guarantee.

These two tricks decrease the amortized time complexity of both Union-Find operations to $O(\alpha(n))$, where $\alpha(n)$ is the inverse Ackermann function. This means that in practice, the **runtime complexiy is ~ constant**.

In [5]:
from __future__ import annotations

class UnionFind:
    
    def __init__(self: UnionFindNaive, n_vertices: int, find_func: str = "iterative") -> None:
        self.parent = {vertex: vertex for vertex in range(1, n_vertices + 1)}
        self.rank = {vertex: 0 for vertex in range(1, n_vertices + 1)}
        if find_func in ("iterative", "recursive"):
            self.find_func = find_func
        else:
            raise ValueError("find_func attribute can only be 'iterative' or 'recursive'")

    def find_iterative(self: UnionFindNaive, v: int) -> None|int:
        root = self.parent[v]
        while root != self.parent[root]:
            root = self.parent[root]
        while v != root:
            tmp = self.parent[v]
            self.parent[v] = root
            v = tmp
        return root
            
    def find_recursive(self: UnionFindNaive, v: int) -> None|int:
        if v != self.parent[v]:
            self.parent[v] = self.find_recursive(self.parent[v])
        return self.parent[v]
    
    def find(self: UnionFindNaive, v: int):
        if self.find_func == "iterative":
            return self.find_iterative(v)
        return self.find_recursive(v)
    
    def union(self: UnionFindNaive, edge: tuple[int, int]) -> bool:
        v1, v2 = edge
        root1, root2 = self.find(v1), self.find(v2)
        if root1 == root2:
            return False
        if self.rank[root1] > self.rank[root2]:
            self.parent[root2] = root1
        elif self.rank[root1] < self.rank[root2]:
            self.parent[root1] = root2
        else:
            self.parent[root1] = root2
            self.rank[root2] += 1
        return True
    
uf = UnionFind(5, "recursive")
uf.union((1, 4))
uf.union((3, 4))


True

# 10 Minimum Spanning Trees

* Tree: undirected graph where there is *exactly one path* between any pair of nodes $\Longleftrightarrow$ acyclic connected undirected graph
* Spanning tree: subgraph of undirected graph $G$ which includes all vertices of G

In [2]:
edges = [
    ("A", "B", 10),
    ("A", "C", 3),
    ("B", "C", 4),
    ("B", "D", 1),
    ("C", "D", 4),
    ("C", "E", 4),
    ("D", "E", 2)
]

# Adjacency list
def build_weighted_adj_list(edges: list[tuple[str, str, int]]) -> dict[str: list[tuple[str, int]]]:
    graph = {}
    for src, dst, distance in edges:
        if src not in graph:
            graph[src] = []
        if dst not in graph:
            graph[dst] = []
        graph[src].append((dst, distance))
        graph[dst].append((src, distance))
    return graph

build_weighted_adj_list(edges)

{'A': [('B', 10), ('C', 3)],
 'B': [('A', 10), ('C', 4), ('D', 1)],
 'C': [('A', 3), ('B', 4), ('D', 4), ('E', 4)],
 'D': [('B', 1), ('C', 4), ('E', 2)],
 'E': [('C', 4), ('D', 2)]}

## 10.a Prim

Greedy algorithm very similay to Dijkstra, it works well with an adjacency list as input.  
  
Core idea: start from any node, then keep track of the spanned nodes (`visited` hashset) and greedily add new edges using a min heap.

In [3]:
import heapq

def mst_prim(edges: list[tuple[str, str, int]]) -> int:
    """Returns the total weight of a minimun spanning tree of a given graph `adj_list` """
    adj_list = build_weighted_adj_list(edges)
    start_node = next(iter(adj_list))
    visited, heap = set(), [[0, start_node]]
    total_weight = 0
    while heap:
        w, v = heapq.heappop(heap)
        if v in visited:
            continue
        total_weight += w
        visited.add(v)
        for neighbor, w in adj_list[v]:
            heapq.heappush(heap, [w, neighbor])
    return total_weight if len(visited) == len(adj_list) else -1

mst_prim(edges)

10

Note for the second implementation: the start node's edges must be pushed to the min heap *before* the main loop.

In [4]:
def mst_prim_edges(edges: list[tuple[str, str, int]]) -> list[tuple[str, str]]:
    """Returns the edges of a minimun spanning tree of a given graph `adj_list`"""
    adj_list = build_weighted_adj_list(edges)
    start_node = next(iter(adj_list))
    visited, heap, mst = set(), [], []

    visited.add(start_node)
    for neighbor, w in adj_list[start_node]:
        heapq.heappush(heap, [w, start_node, neighbor])

    while heap:
        w, src, dst = heapq.heappop(heap)
        if dst in visited:
            continue
        visited.add(dst)
        mst.append([src, dst, w])
        for neighbor, w in adj_list[dst]:
            heapq.heappush(heap, [w, dst, neighbor])
    return mst

mst_prim_edges(edges)

[['A', 'C', 3], ['C', 'B', 4], ['B', 'D', 1], ['D', 'E', 2]]

## 10.b Kruskal

Sort the edges by weights and initialize a UnionFind instance with the appropriate number of vertices. Add the edges one by one, skipping them if they create a cycle. Stop when the expected amount of edges have been found ($V - 1$).

In [8]:
edges = [
    (1, 2, 10),
    (1, 3, 3),
    (2, 3, 4),
    (2, 4, 1),
    (3, 4, 4),
    (3, 5, 4),
    (4, 5, 2)
]

def mst_kruskal(n_vertices: int, edges: list[tuple[int, int, int]]) -> list[tuple[int, int, int]]:
    """Returns the edges of a minimun spanning tree or an empty list if the graph is not connected"""
    edges.sort(key= lambda t: t[-1])
    uf, mst = UnionFind(n_vertices), []
    for u, v, w in edges:
        if not uf.union((u, v)):
            continue
        mst.append((u, v, w))
        if len(mst) == n_vertices - 1:
            return mst
    return []

print(mst_kruskal(5, edges))

[(2, 4, 1), (4, 5, 2), (1, 3, 3), (2, 3, 4)]
